# Libraries

In [1]:
from datetime import datetime
import pandas as pd
import pyodbc

pd.io.formats.excel.ExcelFormatter.header_style = None

# Parameters

In [2]:
today = datetime.today()
year = today.year
week = today.isocalendar()[1]
update_from = '2022-10'
update_to = '2022-10'

path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\MEX\Precios\Data\2022\03 Marzo\Output'
filename = '2022 Marzo_20220304_1542.xlsx'

# Connection

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

## Queries

In [8]:
qprices = '''SELECT  P.Año
       ,P.Semana
       ,C.PaisNombre
       --,C.GrpID
       ,C.GrpNombre
       ,P.CadID
       --,C.CadNombre
       ,P.ProPstID
       ,P.[Precio Base]
  FROM Gnm_MasterOp.dbo.GnmPreciosxCadenaHist AS P
  RIGHT JOIN Gnm_MasterOp.dbo.vw_EstructuraClientesSegPTVTotal AS C
  ON P.CadID = C.CadID
  WHERE C.PaisNombre LIKE '%méx%' AND P.Año = {0} AND P.Semana = {1}
  '''

In [14]:
qproducts = '''SELECT DISTINCT 
          --PA.TipoComNombre AS País
          PR.ProPstCodBarras AS EAN
          ,PR.ProPstNombre AS Descripción
          ,PR.ProPstID
FROM Gnm_MasterOp.dbo.GnmPresentacionesProd AS PR
RIGHT JOIN Gnm_MasterOp.dbo.GnmTiposComProd AS PA
ON PA.TipoComProd = PR.TipoComProd
WHERE PA.TipoComNombre LIKE '%canal%'
ORDER BY ProPstID DESC;
'''

In [10]:
qweeks = '''SELECT SemID
        ,SemAnio
        ,SemNumero
  FROM [Gnm_MasterOp].[dbo].[CatSemanas]
  WHERE (SemAnio BETWEEN {0} AND {1}) AND (SemNumero BETWEEN {2} AND {3})
'''

# Loas Data

## DWH

In [11]:
qprices = qprices.format(year, week)

dprices = pd.read_sql(qprices, conn)

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
qweeks = qweeks.format(
    update_from.split('-')[0], 
    update_to.split('-')[0],
    int(update_from.split('-')[1]),
    int(update_to.split('-')[1])
)

In [15]:
dproducts = pd.read_sql(qproducts, conn)

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [16]:
dweeks = pd.read_sql(qweeks, conn)

c:\Users\jshernandezm\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [19]:
ean_to_update = pd.read_excel(path + '\\' + filename)

In [22]:
len(ean_to_update['Cliente'].unique())

132

In [25]:
len(dprices['GrpNombre'].unique())

31

In [26]:
dprices['GrpNombre'].unique()

array(['Amazon', 'Corporativo Fragua', 'Controladora Comercial Mexicana',
       'Almacen De Drogas', 'Wal-Mart de México', 'Grupo Dorian´s',
       'Grupo Chedraui', 'Grupo Soriana', 'Grupo Coppel', 'Nadro',
       'Liverpool', 'Promesa', 'Sanborns Hermanos', 'Marzam',
       'Supermercados Internacionales Heb', 'Cadena Comercial OXXO',
       'Comercializadora DAX', 'Casa Ley', 'COSTCO',
       'Farmacos Nacionales', 'Grupo Saba', 'Grupo Control', 'Sears',
       'City Club', 'FARMACIAS ABC', 'Farmacias Del Ahorro',
       'Comercial City Fresko', 'Farmacia San Pablo', 'Sams Club',
       'Grupo Benavides', 'FARMACON'], dtype=object)